In [15]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city:str):
    """Get the weather in a given city"""
    print(city)
    return "30 degrees"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather]
)

stream = Runner.run_streamed(agent, "What is the weather in the capitla of Spain?")
async for event in stream.stream_events():
    if event.type=='raw_response_event':
        continue
    elif event.type=="agent_updated_stream_event":
        print("Agent updated to: ", event.new_agent.name)
    elif event.type=="run_item_stream_event":
        if event.item.type=="tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type=="tool_call_output_item":
            print(event.item.output)
        elif event.item.type=="message_output_item":
            print(ItemHelpers.text_message_output(event.item))

    print('='*30)

Agent updated to:  Assistant Agent
{'arguments': '{"city":"Madrid"}', 'call_id': 'call_RH5i7yV2jkHKjJoF3rvM86Kc', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_07c45a96ec199927006997453f8a7c81a1a1577d1e76ff5a80', 'status': 'completed'}
Madrid
30 degrees
The weather in Madrid, the capital of Spain, is currently 30 degrees.


In [19]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city:str):
    """Get the weather in a given city"""
    print(city)
    return "30 degrees"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather]
)

stream = Runner.run_streamed(agent, "What is the weather in the capitla of Spain?")
message = ""
args = ""
async for event in stream.stream_events():
    if event.type=='raw_response_event':
        event_type = event.data.type
        if event_type == "response.output_text.delta":
            message += event.data.delta
            print(message)
        elif event_type == "response.function_call_arguments.delta":
            args += event.data.delta
            print(args)
        elif event_type == "response.completed":
            message = ""
            args = ""
    

{"
{"city
{"city":"
{"city":"Madrid
{"city":"Madrid"}
Madrid
The
The weather
The weather in
The weather in Madrid
The weather in Madrid,
The weather in Madrid, the
The weather in Madrid, the capital
The weather in Madrid, the capital of
The weather in Madrid, the capital of Spain
The weather in Madrid, the capital of Spain,
The weather in Madrid, the capital of Spain, is
The weather in Madrid, the capital of Spain, is currently
The weather in Madrid, the capital of Spain, is currently 
The weather in Madrid, the capital of Spain, is currently 30
The weather in Madrid, the capital of Spain, is currently 30 degrees
The weather in Madrid, the capital of Spain, is currently 30 degrees Celsius
The weather in Madrid, the capital of Spain, is currently 30 degrees Celsius.


### 7.3 Session Memory
* conversation history를 SQLite database에 저장 -> 프로그램이 죽어도 memory는 남음
* 하나의 db -> 여러 대화 처리 가능
* we can manipulate session (대화를 추가, 삭제, 수정 등)

In [25]:
from agents import Agent, Runner, function_tool, SQLiteSession

session = SQLiteSession("user_2", "ai-memory.db")

@function_tool
def get_weather(city:str):
    """Get the weather in a given city"""
    print(city)
    return "30 degrees"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. Use tools when needed to answer questions",
    tools=[get_weather]
)

result = await Runner.run(agent, "What is the weather in the capitla of Spain?", session = session)

print(result.final_output)

Madrid
The weather in Madrid, the capital of Spain, is currently 30 degrees.


In [ ]:
result = await Runner.run(
    agent, 
    "What is my name?", 
    session=session)

print(result.final_output)

await session.add_items([
    {"role":"user", "content":"My name is Sam."}
])


Your name is Sam.


In [ ]:
await session.pop_item() #마지막 아이템을 삭제.

{'id': 'msg_067578737814e92b0069974c2558048190838e378445508e8f',
 'content': [{'annotations': [],
   'text': 'Your name is Sam.',
   'type': 'output_text',
   'logprobs': []}],
 'role': 'assistant',
 'status': 'completed',
 'type': 'message'}

### 7.4 Handoffs
* 다른 에이전트에게 넘기는 것

In [35]:
from agents import Agent, Runner, SQLiteSession

session = SQLiteSession("user_2", "ai-memory.db")

@function_tool
def get_weather(city:str):
    """Get the weather in a given city"""
    print(city)
    return "30 degrees"

geography_agent = Agent(
    name="Geo Expert Agent",
    instructions = "You are a expert in geography. You answer questions related to the.",
    handoff_description="Use this to answer geography-related questions."
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions = "You are a expert in economy. You answer questions related to the.",
    handoff_description="Us this to answer economy-related questions."
)

main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[economics_agent, geography_agent]
)

result = await Runner.run(
    main_agent,  #starting agent
    "Why do countries sell bonds?", 
    session = session)

print(result.last_agent.name)
print(result.final_output)

Economics Expert Agent
Countries sell bonds primarily to raise money to finance various government activities and obligations. Here’s why countries issue and sell bonds:

1. **Government Spending:** Selling bonds allows governments to obtain the funds needed for projects such as building infrastructure (roads, schools, hospitals), funding social programs, or supporting economic growth.

2. **Budget Deficits:** When government spending exceeds its tax revenues, countries sell bonds to cover the deficit without having to raise taxes or cut essential services immediately.

3. **Debt Refinancing:** Governments may issue new bonds to pay off old ones (debt roll-over), helping manage debt payments and extend the maturity of their obligations.

4. **Monetary Policy:** Central banks sometimes buy and sell government bonds to influence money supply and interest rates as part of their monetary policy.

5. **Investor Demand:** Many investors seek low-risk investments, and government bonds are con

### 7.5 Viz and Structured Outputs

In [43]:
from agents import Agent, Runner, SQLiteSession, function_tool, trace
from agents.extensions.visualization import draw_graph
from pydantic import BaseModel

session = SQLiteSession("user_1111111", "ai-memory.db")

class Answer(BaseModel):
    answer: str
    background_explanation: str

@function_tool
def get_weather():
    return "30"

geography_agent = Agent(
    name="Geo Expert Agent",
    instructions = "You are a expert in geography. You answer questions related to the.",
    handoff_description="Use this to answer geography-related questions.",
    tools=[get_weather],
    output_type=Answer,
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions = "You are a expert in economy. You answer questions related to the.",
    handoff_description="Us this to answer economy-related questions."
)

main_agent = Agent(
    name="Main Agent",
    instructions="You are a user facing agent. Transfer to the agent most capable of answering the user's question.",
    handoffs=[economics_agent, geography_agent]
)

# draw_graph(main_agent)
with trace("user_1111111"):
    result = await Runner.run(
        main_agent,  #starting agent
        "What is the capital of Colombia?", 
        session = session)
    result = await Runner.run(
        main_agent,  #starting agent
        "What is the capital of Cambodia?", 
        session = session)
    result = await Runner.run(
        main_agent,  #starting agent
        "What is the capital of Thailand?", 
        session = session)
    

# print(result.last_agent.name)
# print(result.final_output)

### 7.9 Streamlit

In [ ]:
import streamlit as st

if "is_admin" not in st.session_state:
    st.session_state["is_admin"]=False #새로고침돼도 유지되는 변수들

print("running")
st.header("hello!")
name = st.text_input("What is your name?")
if name:
    st.write(f"Hello {name}")
    st.session_state["is_admin"]=True
st.text(st.session_state["is_admin"])